In [ ]:
import pypesto
import pypesto.petab
import pypesto.optimize as optimize
import pypesto.sample as sample
import pypesto.visualize as visualize

import petab
import numpy as np
import scipy
import pickle

from amici.petab_import import import_petab_problem
from amici.petab_objective import simulate_petab

# import to petab
petab_problem = petab.Problem.from_yaml(
    "conversion_reaction/SS_conversion_reaction.yaml")
# import to pypesto
importer = pypesto.petab.PetabImporter(petab_problem) #PetabImporter class-object (subclass of AmiciObjectBuilder)
# create problem
problem = importer.create_problem(guess_steadystate=False) #pypesto.problem class-object

amici_model = import_petab_problem(petab_problem) 

my_internal_amici_objective = importer.create_objective() #pypesto.AmiciObjective class-object

After importing the model, we need to define the objective function. This time we will do it via an external function that will be used then by pyPESTO instead of using the built-in ones.

For numerical reasons we will implement the log likelihood and log prior.

In [ ]:
def log_prior(x): # x[0] = theta, x[1] = c, x[2] = gamma
    """ Log prior function."""
    # assign variables from input x
    offset = x[1]
    precision = x[2] #assuming we receive gamma and not sigma^2
    
    # evaluate log normal-gamma prior
    l_prior = alpha * np.log(beta) + (alpha - 0.5) * np.log(lamda) + 0.5 * (np.log(kappa) - np.log(2) - np.log(np.pi)
                - np.log(scipy.special.gamma(alpha))
    
    return l_prior


def negative_log_posterior(x):
    """ Negative log posterior function."""
    
    # experimental data
    data = np.asarray(petab_problem.measurement_df.measurement)
    
    # simulate model with AMICI
    _simu = my_internal_amici_objective(x, [0], return_dict=True)
    
    # return here the observable
    simulation = _simu['rdatas'][0].y[:, 0]
    
    # evaluate standard log likelihood
    sum = 0
    for k in range(N):
        sum += (y[k] - h[k])**2
    l_llh = N/2 * (np.log(lamda) - np.log(2*np.pi)) - lamda * 0.5 * sum
    
    # evaluate log normal-gamma prior
    l_prior = log_prior(x)
    
    # return NEGATIVE log posterior (required for pyPESTO)
    return -(l_llh + l_prior)


Now that we have the objective function defined, we need to create a pyPESTO problem.

In [ ]:
def standard_sampling():
    """Creates a pyPESTO problem."""
    objective = pypesto.Objective(fun=negative_log_posterior)
    problem = pypesto.Problem(objective=objective,  # objective function
                              lb=[-5, -5, -np.inf, -np.inf],  # lower bounds
                              ub=[5, 5, np.inf, np.inf],  # upper bounds
                              x_names=['k1', 'k2', 'offset', 'sigma'],  # parameter names
                              x_scales=['log', 'log', 'log', 'log'])  # parameter scale
    return problem

In [ ]:
### Prior dependent paramters
# center the scaling parameter
mu=
# std for scaling parameter --> higher = more constrained / lower = more relaxed
alpha=
# center the sigma parameter
beta=
# std for scaling parameter --> higher = more constrained / lower = more relaxed
kappa=

# create the estimation problem
problem = standard_sampling()

# MCMC chain length
n_samples=

# call the sampler of choice
sampler = sample.AdaptiveMetropolisSampler()

Now we can perform the actual sampling.

In [ ]:
# Define number of runs
runs = 1

save_results = False # for testing just set to False

# Loop over n runs
for n in range(runs):
    # set initial random seed
    np.random.seed(n)
    # perform MCMC sampling
    result = sample.sample(problem, n_samples=n_samples, sampler=sampler,
                           x0=np.array([-1.2741, -0.6160, 0.7826, -3.6839]))
    # calculate effective sample size
    sample.effective_sample_size(result=result)

    # save the results as a pickle object
    if save_results:
        with open('result_SS_'+str(n)+'.pickle', 
                  'wb') as result_file:
            pickle.dump(result.sample_result, result_file)
            result_file.close()

There are some built-in visualization functions that one can use.

In [ ]:
# Visualize the log posterior trace
ax = visualize.sampling_fval_trace(result, size=(12,5), full_trace=True)
# Visualize the parameter trace
ax = visualize.sampling_parameters_trace(result, use_problem_bounds=False, full_trace=True, size=(12,5))
# Visualize the one-dimensional marginals --> Important!
ax = visualize.sampling_1d_marginals(result, size=(12,5))